## Exploration of large scientific dataset with OpenVisus and PyVista

### OpenViSUS: install package

To execute this jupyter notebook have to install the package "OpenViSUS"

In conda environments you can use:
```
conda install -c visus OpenVisus
python -m OpenVisus configure
```
Alternatively, you can install it via the Anaconda package manager after adding the "visus" channel to the environment.

In pip envirinments you can use:
```
python -m pip install OpenVisus

python -m OpenVisus configure
```
Note: ignore errors during the "configure" process

### OpenViSUS: read from a remote dataset

In [1]:
%matplotlib notebook

import os,sys

# Here are commands to install a package (OpenVisus) directly from a jupyter notebook
# after you install those once you can comment those comment
# !{sys.executable} -m pip install itkwidgets
#!{sys.executable} -m OpenVisus configure

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv

import OpenVisus as ov

# Enable I/O component of OpenVisus
ov.DbModule.attach()

In [2]:
#function to plot the image data with matplotlib
#optional parameters: colormap, existing plot to reuse (for more interactivity)
def showData2D(data, cmap=None, plot=None):
    if len(data.shape)==3 and data.shape[0]==1: data=data[0,:,:]
    if len(data.shape)==3 and data.shape[1]==1: data=data[:,0,:]   
    if len(data.shape)==3 and data.shape[2]==1: data=data[:,:,0]
    if(plot==None or cmap!=None):
        fig=plt.figure(figsize = (7,5))
        plot = plt.imshow(data, origin='lower', cmap=cmap)
        plt.show()
        return plot
    else:
        plot.set_data(data)
        plt.show()
        return plot

def showData(data, cmap=None, pl=None, opacity='linear'):
    
    data = pv.wrap(data)
    if pl == None:
        pl = pv.Plotter()
    #     pl.clear_actors()
        actor = pl.add_mesh(data, cmap=cmap, opacity=opacity)
        pl.show()
        return pl
    else:
        pl.clear_actors()
        actor = pl.add_mesh(data, cmap=cmap, opacity=opacity)
        pl.show()
        return pl
#     if len(data.shape)==3 and data.shape[0]==1: data=data[0,:,:]
#     if len(data.shape)==3 and data.shape[1]==1: data=data[:,0,:]   
#     if len(data.shape)==3 and data.shape[2]==1: data=data[:,:,0]
#     if(plot==None or cmap!=None):
#         fig=plt.figure(figsize = (7,5))
#         plot = plt.imshow(data, origin='lower', cmap=cmap)
#         plt.show()
#         return plot
#     else:
#         plot.set_data(data)
#         plt.show()
#         return plot
    

### Navigate time and resolution

In [3]:
# select a remote dataset (satellite imagery from NASA)
dataset=ov.LoadDataset("https://atlantis.sci.utah.edu/mod_visus?dataset=BlueMarble")
# this doesn't actually fetch any data, only metadata

# what is the size of this dataset ?
# the logic box contains the extent of the dataset on the different axis
#dataset.getLogicBox().toString()
print(dataset.getDatasetBody().toString())

<dataset url="https://atlantis.sci.utah.edu/mod_visus?dataset=BlueMarble" cache_dir="" typename="IdxDataset">
	<idxfile>
		<version value="6" />
		<bitmask value="V001010101010101010101010101010101" />
		<box value="0 86400 0 43200" />
		<bitsperblock value="16" />
		<blocksperfile value="256" />
		<block_interleaving value="0" />
		<filename_template value="./bluemarble-compressed/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="time%02d/" />
		<physic_box value="0 5760 0 2881" />
		<field name="data" description="" index="" default_compression="zip" default_layout="" default_value="0" filter="" dtype="uint8[3]" />
		<timestep from="0" to="11" step="1" />
	</idxfile>
</dataset>


In [4]:
# what is the maximum resolution ? 
# NOTE: don't use large values of the resolution for large query (you will be fetching too much data)
dataset.getMaxResolution()

33

In [5]:
# function to read data from a remote dataset
# optional parameters: timestep, field (variable in the dataset), logic_box (bounding box of the query), resolution

# Note: the resolution value could sometime fetch a dataset with the wrong aspect ratio, 
# this because in the IDX format we double the size at each resolution on only one of the axis at a time

data = dataset.read(time=0, max_resolution=21)
data.shape
# data = pv.wrap(data)
# data.n_cells

(675, 1350, 3)

In [6]:
# Plot the 2D image using matplotlib with different time and resolutions

interact(
    lambda time,resolution: showData2D(dataset.read(time=time,max_resolution=resolution)),
    time=widgets.IntSlider(value=0,min=0,max=11,step=1), 
    resolution=widgets.IntSlider(value=9,min=1,max=21,step=2))

interactive(children=(IntSlider(value=0, description='time', max=11), IntSlider(value=9, description='resoluti…

<function __main__.<lambda>(time, resolution)>

In [7]:
# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape + 1 because we want to inject our values on
#   the CELL data
grid.dimensions = np.array((data.shape[1], data.shape[0], 1))
# print(grid.dimensions, grid.n_cells)

# Edit the spatial reference
#grid.origin = (100, 33, 55.6)  # The bottom left corner of the data set
#grid.spacing = (1, 5, 2)  # These are the cell sizes along each axis

# Add the data values to the cell data
grid.point_data["values"] = data.reshape(-1, data.shape[-1]) # Flatten the array!

# grid
# Now plot the grid!
grid.plot(rgb=True, cpos="xy")

Widget(value="<iframe src='http://localhost:60933/index.html?ui=P_0x150be623880_0&reconnect=auto' style='width…

In [8]:
def show_data(data, pl=None):
    grid = pv.UniformGrid()
    grid.dimensions = np.array((data.shape[1], data.shape[0], 1))
    grid.point_data["values"] = data.reshape(-1, data.shape[-1])
    
    if pl == None:
        pl = pv.Plotter()
    else:
        pl.clear_actors()
    
    actor = pl.add_mesh(grid, rgb=True)
    
    pl.show(cpos="xy")
    pl.reset_camera()
    return pl

In [9]:
# create a plot for our data
# myplot = show_data(dataset.read(time=0, max_resolution=21))
pl = pv.Plotter()
# reuse the plot with an interact for varying time and resolution values
interact(
    lambda time,resolution: show_data(dataset.read(time=time,max_resolution=resolution), pl=pl),
    time=widgets.IntSlider(value=0,min=0,max=11,step=1), 
    resolution=widgets.IntSlider(value=9,min=1,max=21,step=2))

interactive(children=(IntSlider(value=0, description='time', max=11), IntSlider(value=9, description='resoluti…

<function __main__.<lambda>(time, resolution)>

### Interactive analysis

In [10]:
# Open an aerial dataset from the National Ecology Observatory Network 
dataset=ov.LoadDataset("https://atlantis.sci.utah.edu/mod_visus?dataset=neon_redb")
print(dataset.getDatasetBody().toString(), dataset.getMaxResolution())

<dataset url="https://atlantis.sci.utah.edu/mod_visus?dataset=neon_redb" cache_dir="" typename="IdxDataset">
	<idxfile>
		<version value="6" />
		<bitmask value="V0101010101010101010101010101010101" />
		<box value="0 80000 0 100000" />
		<bitsperblock value="16" />
		<blocksperfile value="512" />
		<block_interleaving value="0" />
		<filename_template value="./2017_REDB_1/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="" />
		<field name="DATA" description="" index="" default_compression="" default_layout="" default_value="0" filter="" dtype="uint8[3]" />
		<timestep when="0" />
	</idxfile>
</dataset> 34


In [11]:
data = dataset.read(max_resolution=22)
# showData(data)

grid = pv.UniformGrid()
grid.dimensions = np.array((data.shape[1], data.shape[0], 1))
grid.point_data["values"] = data.reshape(-1, data.shape[-1])
grid.plot(rgb=True, cpos="xy")

Widget(value="<iframe src='http://localhost:60933/index.html?ui=P_0x150be5efa90_2&reconnect=auto' style='width…

In [12]:
# what's the name of the field we are looking at?
dataset.getField().name

'DATA'

In [13]:
# what's the datatype fo this field?
dataset.getField('DATA').dtype.toString()

''

In [14]:
# what's the size of the data fetched? (note: there are three channels, RGB)
data.shape

(1563, 1250, 3)

In [15]:
# make a "grey scale" version of the data
# from Matlab "rgb2gray" 0.2989 * R + 0.5870 * G + 0.1140 * B (standard ITU-R BT.601-7)

R,G,B=(0.2989*data[:,:,0], 0.5870*data[:,:,1], 0.1140*data[:,:,2])
grey_data=R+G+B
grey_data = np.reshape(grey_data, (1563, 1250, 1))
print(grey_data)
print(type(grey_data))

[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]]
<class 'numpy.ndarray'>


In [16]:
pv.wrap(grey_data)

UniformGrid (0x150c0451a00)
  N Cells:      1950938
  N Points:     1953750
  X Bounds:     0.000e+00, 1.562e+03
  Y Bounds:     0.000e+00, 1.249e+03
  Z Bounds:     0.000e+00, 0.000e+00
  Dimensions:   1563, 1250, 1
  Spacing:      1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:     1

In [17]:
# show data using a grey scale colormap
showData(grey_data, cmap=plt.get_cmap("Greys"))

Widget(value="<iframe src='http://localhost:60933/index.html?ui=P_0x150c03e61f0_3&reconnect=auto' style='width…

In [20]:
# make a threshold function to show which "pixel" is above a certain values
def threshold(data, t):
    return data > t

In [23]:
test = threshold(grey_data, 200)
print(test)
showData(threshold(grey_data, 200), cmap=plt.get_cmap("Greys"))

<class 'numpy.ndarray'>
[[[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]

 [[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]

 [[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]

 ...

 [[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]

 [[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]

 [[False]
  [False]
  [False]
  ...
  [False]
  [False]
  [False]]]
<class 'numpy.ndarray'>


RuntimeError: Transfer function cannot have more values than `n_colors`. This has 256 elements

In [ ]:
# make the threshold exploration interactive
# myplot = showData(threshold(grey_data,t=150))
pl = pv.Plotter()
interact(
    lambda thr: showData(threshold(grey_data,t=thr), cmap=plt.get_cmap("Greys"), pl=pl),
    thr=widgets.IntSlider(value=np.mean(grey_data),min=np.min(grey_data),max=np.max(grey_data),step=1))

interactive(children=(IntSlider(value=96, description='thr', max=254), Output()), _dom_classes=('widget-intera…

<function __main__.<lambda>(thr)>

### Working with 3D dataset

In [ ]:
dataset=ov.LoadDataset("https://atlantis.sci.utah.edu/mod_visus?dataset=borg")
data = dataset.read(time=10, max_resolution=24)
showData(data, opacity=[0.2,0.8,1.0])

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
dataset=ov.LoadDataset("https://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1")
# how big is the dataset ?
print(dataset.getDatasetBody().toString())

<dataset url="https://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1" typename="IdxDataset">
	<idxfile>
		<version value="6" />
		<bitmask value="V012012012012012012012012012012012" />
		<box value="0 2048 0 2048 0 2048" />
		<bitsperblock value="16" />
		<blocksperfile value="256" />
		<block_interleaving value="0" />
		<filename_template value="./visus/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="" />
		<field name="DATA" description="" index="" default_compression="zip" default_layout="hzorder" default_value="0" filter="" dtype="uint8" />
		<timestep when="0" />
	</idxfile>
</dataset>


In [ ]:
# make a query to fetch a slice of this 3D dataset (in the middle of the 3rd axis)
data=dataset.read(x=[0,2048],y=[0,2048],z=[1024,1025],max_resolution=21)
print(data.shape)
showData(data, opacity=[1.0])

(1, 128, 128)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
data=dataset.read(max_resolution=21)
print(data.shape)

# data = pv.wrap(data)
showData(data)

(72, 72, 72)


ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
showData(data, opacity='sigmoid')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# loading a large 2D microscopy dataset showing the retina of a rabbit
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0], [131072, 131072])

In [ ]:
# get default field name
dataset.getField().name

'EM'

In [ ]:
data=dataset.read(max_resolution=22)
showData(data.reshape(data.shape + (1,)))

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# open a 3D version of this dataset
dataset=ov.LoadDataset("http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit3d")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0, 0], [131072, 131072, 341])

In [ ]:
# almost 6TB size, can we still visualize it on this browser?
131073*131073*342/(1024*1024*1024)

5472.083496412262

In [ ]:
data=dataset.read(max_resolution=21)
data = pv.wrap(data)

pl = pv.Plotter()
actor = pl.add_volume(data)
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

### Voxel spacing

Microscopy data or CT scan often have different resolution along the axis, in order to make a realistic visualization we need to use correct voxel spacing

In [ ]:
data=dataset.read(max_resolution=21)
grid = pv.UniformGrid()
grid.dimensions = np.array((data.shape[2],data.shape[1], data.shape[0]))
grid.point_data["values"] = data.ravel()

In [ ]:
grid.spacing = (1.0, 1.0, 0.1)

In [ ]:
pl = pv.Plotter()
actor = pl.add_volume(grid)
pl.show(cpos='xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
# we open a metallic foam dataset
dataset=ov.LoadDataset("https://atlantis.sci.utah.edu/mod_visus?dataset=foam")
# how big is the dataset ?
# the logic box contains the extent of the dataset on the different axis
dataset.getLogicBox()

([0, 0, 0], [1055, 1024, 1024])

In [ ]:
# Visualize and explore the dataset 
# How can we evaluate the density of material?

data=dataset.read(max_resolution=19)

wrapped_data = pv.wrap(data)
pl = pv.Plotter()
actor = pl.add_volume(wrapped_data)
pl.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [ ]:
wrapped_data

UniformGrid (0x1302c3340)
  N Cells:      257985
  N Points:     270336
  X Bounds:     0.000e+00, 6.300e+01
  Y Bounds:     0.000e+00, 6.300e+01
  Z Bounds:     0.000e+00, 6.500e+01
  Dimensions:   64, 64, 66
  Spacing:      1.000e+00, 1.000e+00, 1.000e+00
  N Arrays:     1

In [ ]:
# Evaluate the density of material 
mat_interface_val = 10000
count = (data > mat_interface_val).sum()
density = count/(data.shape[0]*data.shape[1]*data.shape[2])
density


0.3226355350378788

In [ ]:
def density_res(res):
    data=dataset.read(max_resolution=res)
    count = (data > mat_interface_val).sum()
    density = count/(data.shape[0]*data.shape[1]*data.shape[2])
    return density

In [ ]:
density_res(21)

0.3249557957504735

In [ ]:
density_res(20)

0.32427238695549243

In [ ]:
density_res(18)

0.3226355350378788

In [ ]:
density_res(17)

0.04532137784090909

In [ ]:
density_res(15)

0.3151041666666667

In [ ]:
density_res(10)

0.3107638888888889

In [ ]:
density_res(23)

0.32539067123875476